# Web Scraping & NLP in Python
Available in: https://www.datacamp.com/community/tutorials/web-scraping-python-nlp

## 1. State Your Question
What are the most frequent words in the novel Moby Dick and how often do they occur?

## 2. Get Your Data
Your raw data is the text of Melville's novel Moby Dick. How would you go about getting the text of this ~800 word book into Python?

Well, there are several ways to do this but first realize that the text is freely available online at Project Gutenberg. Let's head there, try to find Moby Dick and then store the relevant URL in your Python namespace:

In [ ]:
# Store url
url = 'https://www.gutenberg.org/files/2701/2701-h/2701-h.htm'

Now that you have the URL, you need to fetch the HTML of the website. Note that HTML stands for Hypertext Markup Language and is the standard markup language for the web. You're going to use requests to do this, one of the most popular and useful Python packages out there. 

You'll be making a GET request from the website, which means you're getting data from it. This is what you're doing through a browser when visiting a webpage using a browser. There are other types of requests, such as POST requests, but we won't concern ourselves with them here.

**requests** make this easy with its *get* function. Make the request here and check the object type returned.

In [ ]:
# Import `requests`
import requests

# Make the request and check object type
r = requests.get(url)
type(r)

This is a Response object. You can see in the requests kickstart guide that a Response object has an attribute text that allows you to get the HTML from it! Let's do this and print the HTML to check it out:

In [ ]:
# Extract HTML from Response object and print
html = r.text
print(html)

OK! This HTML is not quite what you want. However, it does contain what you want: the text of Moby Dick. What you need to do now is wrangle this HTML to extract the novel.

## 3. Wrangle the Data to Answer the Question
### Part 1: Get the Text from the HTML

Firstly, a word on the name of the package: Beautiful Soup? In web development, the term "tag soup" refers to structurally or syntactically incorrect HTML code written for a web page. What Beautiful Soup does best is to make tag soup beautiful again and to extract information from it with ease! In fact, the main object created and queried when using this package is called BeautifulSoup. After creating the soup, we can use its .get_text() method to extract the text.

In [ ]:
# Import BeautifulSoup from bs4
from bs4 import BeautifulSoup


# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, "html5lib")
type(soup)

From these soup objects, you can extract all types of interesting information about the website you're scraping, such as title:

In [ ]:
# Get soup title
soup.title

Or the title as a string:

In [ ]:
# Get soup title as string
soup.title.string

Or all URLs found within a page’s < a > tags (hyperlinks):

In [ ]:
# Get hyperlinks from soup and check out first several
soup.findAll('a')[:8]

What you want to do is to extract the text from the soup and there's a super helpful .get_text() method precisely for this.

Get the text, print it out and have a look at it. Is it what you want?

In [ ]:
# Get the text out of the soup and print it
text = soup.get_text()
print(text)

Notice that this is now nearly what you want.

It is the text of the novel with some unwanted stuff at the start and some unwanted stuff at the end. You could remove it if you wanted. However, this content is so much smaller in amount than the text of Moby Dick that, to a first approximation, it is fine to leave in and this will be the approach here. To get robust results, I'd suggest removing it.

Now that you have the text of interest, it's time for you to count how many times each word appears and to plot the frequency histogram that you want: Natural Language Processing to the rescue!

### Part 2: Extract Words from your Text with NLP
You'll now use nltk, the Natural Language Toolkit, to

- Tokenize the text (fancy term for splitting into tokens, such as words)

- Remove stopwords (words such as 'a' and 'the' that occur a great deal in ~ nearly all English language texts.

Step 1: Tokenize
You want to tokenize your text, that is, split it into a list a words. Essentially, you want to split off the parts off the text that are separated by whitespaces.

To do this, you're going to use a powerful tool called regular expressions. A regular expression, or regex for short, is a sequence of characters that define a search pattern. They are notoriously confusing and best introduced by example.

You have the string 'peter piper picked a peck of pickled peppers' and you want to extract from the list of all words in it that start with a 'p'.
The regular expression that matches all words beginning with 'p' is 'p\w+'. Let's unpack this:

- the 'p' at the beginning of the regular expression means that you'll only match sequences of characters that start with a 'p';

- the '\w' is a special character that will match any alphanumeric A-z, a-z, 0-9, along with underscores;

The '+' tells you that the previous character in the regex can appear as many times as you want in strings that you;re trying to match. This means that '\w+' will match arbitrary sequences of alphanumeric characters and underscores.
Put this all together and the regular expression 'p\w+' will match all substrings that start with a 'p' and are followed by alphanumeric characters and underscores. In most English language texts that make sense, this will correspond to words beginning with 'p'.

You'll now use the built-in Python package re to extract all words beginning with 'p' from the sentence 'peter piper picked a peck of pickled peppers' as a warm-up.

In [ ]:
# Import regex package
import re

# Define sentence
sentence = 'peter piper pick a peck of pickled peppers'

# Define regex
ps = 'p\w+'


# Find all words in sentence that match the regex and print them
re.findall(ps, sentence)

This looks pretty good. Now, if 'p\w+' is the regex that matches words beginning with 'p', what's the regex that matches all words?

It's your job to now do this for our toy Peter Piper sentence above.

In [ ]:
# Find all words and print them
re.findall('\w+', sentence)

Now you can do the same with text, the string that contains Moby Dick:

In [ ]:
# Find all words in Moby Dick and print several
tokens = re.findall('\w+', text)
tokens[:8]

Note that there is also a way to do this with nltk, the Natural Language Toolkit:

In [ ]:
# Import RegexpTokenizer from nltk.tokenize
from nltk.tokenize import RegexpTokenizer

# Create tokenizer
tokenizer = RegexpTokenizer('\w+')

# Create tokens
tokens = tokenizer.tokenize(text)
tokens[:8]

OK! You're nearly there. Note, though, that in the above, 'Or' has a capital 'O' and that in other places it may not but both 'Or' and 'or' you will want to count as the same word. For this reason, you will need to build a list of all words in Moby Dick in which all capital letters have been made lower case. You'll find the string method .lower() handy:

In [ ]:
# Initialize new list
words = []

# Loop through list tokens and make lower case
for word in tokens:
    words.append(word.lower())

# Print several items from list as sanity check
words[:8]

Step 2: Remove stop words
It is common practice to remove words that appear alot in the English language such as 'the', 'of' and 'a' (known as stopwords) because they're not so interesting. For more on all of these techniques, check out our Natural Language Processing Fundamentals in Python course.

The package nltk has a list of stopwords in English which you'll now store as sw and of which you'll print the first several elements.

If you get an error here, run the command nltk.download('stopwords') to install the stopwords on your system.

In [ ]:
# Import nltk
import nltk

# Get English stopwords and print some of them
sw = nltk.corpus.stopwords.words('english')
sw[:5]

You want the list of all words in words that are not in sw. One way to get this list is to loop over all elements of words and add the to a new list if they are not in sw:

In [ ]:
# Initialize new list
words_ns = []

# Add to words_ns all words that are in words but not in sw
for word in words:
    if word not in sw:
        words_ns.append(word)

# Print several list items as sanity check
words_ns[:5]

## 4. Answer your question
Our question was 'What are the most frequent words in the novel Moby Dick and how often do they occur?'

You can now plot a frequency histogram of words in Moby Dick in two line of code using nltk. To do this:

- You create a frequency distribution object using the function nltk.FreqDist();

- You use the plot() method of the resulting object.

In [ ]:
#Import datavis libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Figures inline and set visualization style
%matplotlib inline
sns.set()

# Create freq dist and plot
freqdist1 = nltk.FreqDist(words_ns)
freqdist1.plot(25)

## 5. Present Your Solution
The cool thing is that, in using nltk to answer our question, we actually already presented our solution in a manner that can be communicated to other: a frequency distribution plot! You can read off the most common words, along with their frequency. For example, 'whale' is the most common word in the novel (go figure), excepting stopwords, and it occurs a whopping >1200 times!